# Anomaly Detection

### CSEC620 - Project 5 - Mehul Sen



### Import Required Libraries

In [1]:
# Import numpy for data handling, and basic math calculations
import numpy as np
import pandas as pd

from sklearn.covariance import EllipticEnvelope
from sklearn.linear_model import SGDOneClassSVM
from sklearn.ensemble import IsolationForest
from sklearn.decomposition import PCA
from sklearn.metrics import accuracy_score, classification_report, roc_auc_score

import torch
from torch.nn import Module, ReLU, Linear, Sequential, Sigmoid
from torch.nn.functional import binary_cross_entropy, mse_loss
from torch.utils.data import DataLoader, TensorDataset

import xgboost as xgb

## Preprocessing the Dataset
The preprocessing involves splitting the dataset into its data and label counterparts. Additionally, the preprocessing involves extracting the following columns from the data: `processId`, `parentProcessId`, `userId`, `mountNamespace`, `eventId`, `argsNum`, and `returnValue`. These are used as features for model training and testing.

The following is the feature transformation performed on the dataset. This is done exactly as described in the benchmarks presented by the paper.
- `processId` and `parentProcessId` are transformed: values 0, 1, 2 are mapped to 0 (possibly representing OS processes), and all other values are mapped to 1 (non-OS processes).
- `userId` is transformed: values less than 1000 are mapped to 0, and values 1000 and above are mapped to 1. This could be distinguishing between system users (typically having lower user IDs) and regular users.
- `mountNamespace` is transformed: a specific value (4026531840) is mapped to 0, and all other values are mapped to 1. This might differentiate between a specific mount namespace (perhaps the OS's) and others.
- `eventId` is left unchanged, but the comment indicates consideration for its maximum value, which is not implemented in the code.
- `returnValue` is transformed into three categories: 0 for a return value of 0 (indicating success), 1 for positive return values (success with a value), and 2 for negative return values (error).

In [2]:
# Load the dataset
training_dataset = pd.read_csv('./Dataset/archive/labelled_training_data.csv')
test_dataset = pd.read_csv('./Dataset/archive/labelled_testing_data.csv')

# Function to Preprocess the dataset
def preprocess_data(data):
    labels = pd.DataFrame(data[["sus"]])
    data = pd.DataFrame(data[["processId", "parentProcessId", "userId", "mountNamespace", "eventId", "argsNum", "returnValue"]])
    data["processId"] = data["processId"].map(lambda x: 0 if x in [0, 1, 2] else 1)  # Map to OS/not OS
    data["parentProcessId"] = data["parentProcessId"].map(lambda x: 0 if x in [0, 1, 2] else 1)  # Map to OS/not OS
    data["userId"] = data["userId"].map(lambda x: 0 if x < 1000 else 1)  # Map to OS/not OS
    data["mountNamespace"] = data["mountNamespace"].map(lambda x: 0 if x == 4026531840 else 1)  # Map to mount access to mnt/ (all non-OS users) /elsewhere
    data["eventId"] = data["eventId"]
    data["returnValue"] = data["returnValue"].map(lambda x: 0 if x == 0 else (1 if x > 0 else 2))  # Map to success/success with value/error
    return data, labels['sus']

# Preprocess the dataset
training_data, training_labels = preprocess_data(training_dataset)
test_data, test_labels = preprocess_data(test_dataset)

## Method 1: Robust Covariance
To perform anomaly detection, the first model used is Robust Covariance. This is achieved using the `EllipticEnvelope` model from the `sklearn` library, which is a method for detecting outliers in a Gaussian distributed dataset.

### Description
The `EllipticEnvelope` model with a contamination factor of 0.05 is initialized, which is an estimate of the proportion of outliers in the data. The `random_state` parameter is set to 42, ensuring that the results are reproducible. The model is then fitted to the training data using the `fit` method. After training the model, it is used to calculate the Mahalanobis distances of the test data. The Mahalanobis distance is a measure of the distance between a point and a distribution, not dependent on the scale of measurements. The `mahalanobis` method of the model returns these distances. An anomaly threshold is then set at the 99th percentile of the Mahalanobis distances. This means that any data point with a Mahalanobis distance greater than this threshold is considered an anomaly. Predictions are then made based on this threshold: if the Mahalanobis distance of a data point is greater than the threshold, it is labeled as an anomaly (1), otherwise, it is labeled as normal (0).

### Performance
On testing the model achieves AUCROC of 0.496 which is very close to the result reported in the paper(0.519).
```
Accuracy: 0.09260347044722095
AUCROC: 0.49653769660705876
Report:               precision    recall  f1-score   support

           0       0.09      0.99      0.17     17508
           1       0.48      0.00      0.00    171459

    accuracy                           0.09    188967
   macro avg       0.29      0.50      0.08    188967
weighted avg       0.45      0.09      0.02    188967
```


In [3]:
# NOTE: Approximately takes 1 minute to run

# Initialize the model
robust_covariance_model = EllipticEnvelope(contamination=0.05, random_state=42)

# Fit the model using the training data
robust_covariance_model.fit(training_data)

# Predict using the test data with Mahalanobis distance
mahalanobis_distances = robust_covariance_model.mahalanobis(test_data)
anomaly_threshold = np.percentile(mahalanobis_distances, 99)
test_predictions = [1 if y > anomaly_threshold else 0 for y in mahalanobis_distances]

# Print the results
print("Accuracy:", accuracy_score(test_labels, test_predictions))
print("AUCROC:", roc_auc_score(test_labels, test_predictions))
print("Report:", classification_report(test_labels, test_predictions))

Accuracy: 0.09260347044722095
AUCROC: 0.49653769660705876
Report:               precision    recall  f1-score   support

           0       0.09      0.99      0.17     17508
           1       0.48      0.00      0.00    171459

    accuracy                           0.09    188967
   macro avg       0.29      0.50      0.08    188967
weighted avg       0.45      0.09      0.02    188967



## Method 2: One-Class Support Vector Machine
The second model used is the one-class support vector machine, this discriminates between in-distribution data and out-of-distribution data to perform anomaly detection.

### Description
SGDOneClassSVM is a variant of One-Class SVM that is trained using Stochastic Gradient Descent (SGD). This makes it more efficient for larger datasets. Additionally, a `WhitenedBenchmark` model is used that applies Principal Component Analysis (PCA) with whitening to the data before passing it to the base model. Whitening is a preprocessing step that makes the input features less correlated and have the same variance.

The `SGDOneClassSVM` model is initialized with a random state of 42 for reproducibility. This model is then passed to the `WhitenedBenchmark` model as the base model. This then gets fitted to the training data. After training the model, it is used to make predictions on the test data. The `predict` method of the model returns 1 for inliers and -1 for outliers. These prediction labels are then converted to 0 for inliers and 1 for outliers using a list comprehension.

### Performance
On testing, the model achieves AUCROC of 0.598 which is very close to the results reported in the paper(0.605).
```
Accuracy: 0.8781903718638704
AUCROC: 0.5984245500758777
Report:               precision    recall  f1-score   support

           0       0.31      0.26      0.28     17508
           1       0.93      0.94      0.93    171459

    accuracy                           0.88    188967
   macro avg       0.62      0.60      0.61    188967
weighted avg       0.87      0.88      0.87    188967
```


In [5]:
# Define WhitenedBenchmark for Data Whitening through PCA
class WhitenedBenchmark():
    def __init__(self, base_model):
        self.clf = base_model
        self.pca = PCA(whiten=True)

    def decision_function(self, X):
        return self.clf.decision_function(self.pca.transform(X))

    def fit(self, X):
        self.pca = self.pca.fit(X)
        self.clf = self.clf.fit(self.pca.transform(X))

    def predict(self, X):
        return self.clf.predict(self.pca.transform(X))

In [5]:
# NOTE: Approximately takes 5 seconds to run

# Initialize the model
ocsvm_model = SGDOneClassSVM(random_state=42)
whitened_ocsvm_model = WhitenedBenchmark(ocsvm_model)

# Fit the model using the training data
whitened_ocsvm_model.fit(training_data)

# Predict using the test data
test_predictions = whitened_ocsvm_model.predict(test_data)
test_predictions = [0 if y == 1 else 1 for y in test_predictions] # Convert 1 to 0 and -1 to 1

# Print the results
print("Accuracy:", accuracy_score(test_labels, test_predictions))
print("AUCROC:", roc_auc_score(test_labels, test_predictions))
print("Report:", classification_report(test_labels, test_predictions))

Accuracy: 0.8781903718638704
AUCROC: 0.5984245500758777
Report:               precision    recall  f1-score   support

           0       0.31      0.26      0.28     17508
           1       0.93      0.94      0.93    171459

    accuracy                           0.88    188967
   macro avg       0.62      0.60      0.61    188967
weighted avg       0.87      0.88      0.87    188967



## Method 3: Isolation Forest
This model uses a small set of discrete features available and the conspicous nature of attacks to identify anomalous behaviour within the data, making it the best at differentiating suspicious events from benign events.

### Description
The `IsolationForest` model is an unsupervised learning algorithm that isolates anomalies instead of profiling normal data points. It's useful for detecting anomalies in high-dimensional datasets. The `IsolationForest` model is initialized with a random state of 42 for reproducibility. This model is then passed to the `WhitenedBenchmark` model as the base model which is then fitted to the training data. After training the model, it is used to make predictions on the test data. The `predict` method of the model returns 1 for inliers and -1 for outliers. These prediction labels are then converted to 0 for inliers and 1 for outliers using a list comprehension.

### Performance
On testing, the model achieves AUCROC of 0.770 which is close to the results reported in the paper(0.850).
```
Accuracy: 0.8805452803928728
AUCROC: 0.7707815144740071
Report:               precision    recall  f1-score   support

           0       0.41      0.64      0.50     17508
           1       0.96      0.91      0.93    171459

    accuracy                           0.88    188967
   macro avg       0.68      0.77      0.71    188967
weighted avg       0.91      0.88      0.89    188967
```

In [6]:
# NOTE: Approximately takes 5 seconds to run

# Initialize the model
isolation_forest_model = IsolationForest(random_state=42)
whitened_model = WhitenedBenchmark(isolation_forest_model)

# Fit the model using the training data
whitened_model.fit(training_data)

# Predict using the test data
test_predictions = whitened_model.predict(test_data)
test_predictions = [0 if y == 1 else 1 for y in test_predictions]  # Adjust to match label encoding

# Print the results
print("Accuracy:", accuracy_score(test_labels, test_predictions))
print("AUCROC:", roc_auc_score(test_labels, test_predictions))
print("Report:", classification_report(test_labels, test_predictions))

Accuracy: 0.8805452803928728
AUCROC: 0.7707815144740071
Report:               precision    recall  f1-score   support

           0       0.41      0.64      0.50     17508
           1       0.96      0.91      0.93    171459

    accuracy                           0.88    188967
   macro avg       0.68      0.77      0.71    188967
weighted avg       0.91      0.88      0.89    188967



## Method 4: Variational Auto Encoder(VAE) + Density of States Estimate(DoSE) Support Vector Machine(SVM)
Another method for anomaly detection is by using a Variational Auto Encoder(VAE) to model the observations as a product of categorical distributions, which is then used by Density of State Estimates(DoSE) through a linear one-class SVM trained using SGD to identify anomalies.

### Description
GitHub Copilot: The VAE model is initialized alongside an Adam optimizer with a learning rate of 0.001. The VAE model consists of an encoder and a decoder. The encoder transforms the input data into a latent space representation, and the decoder reconstructs the input data from the latent space representation. This model is trained for 5 epochs using the `train_vae` function. After training, a one-class SVM is trained on the latent representations of the training data using the `train_dose_svm` function. The one-class SVM is used for anomaly detection, where the goal is to learn a boundary around normal data in the latent space, and classify data points that fall outside this boundary as anomalies. Lastly the `get_test_predictions` function is used to generate predictions on the test data using the VAE model and the one-class SVM.

### Performance
On testing, the model achieves AUCROC value of 0.735 which is very close to the results reported in the paper(0.698).
NOTE: While I did use random_seed 42, and removed shuffling for data loaders, the results for this model are different when trained, while they are generally close to the results reported in the paper, there are some instances where they can be much different.
```
Epoch 0: Loss = 0.6435845868719934
Epoch 1: Loss = 0.623629160491492
Epoch 2: Loss = 0.6235394135073636
Epoch 3: Loss = 0.6235290576831732
Epoch 4: Loss = 0.6235514352574132
Accuracy:  0.9147099758158832
AUCROC:  0.7355800062161345
Classification Report:
               precision    recall  f1-score   support

         0.0       0.54      0.52      0.53     17508
         1.0       0.95      0.96      0.95    171459

    accuracy                           0.91    188967
   macro avg       0.75      0.74      0.74    188967
weighted avg       0.91      0.91      0.91    188967
```

In [7]:
# Define VAE Class that learns a latent representation of the data
class VAE(Module):
    def __init__(self, input_dim=7, latent_dim=2, hidden_dims=[64, 32], activation_fn=ReLU()):
        super(VAE, self).__init__()
        # Encoder
        encoder_layers = [Linear(input_dim, hidden_dims[0]), activation_fn]
        for i in range(1, len(hidden_dims)):
            encoder_layers += [Linear(hidden_dims[i-1], hidden_dims[i]), activation_fn]
        self.encoder = Sequential(*encoder_layers)
        self.fc_mu = Linear(hidden_dims[-1], latent_dim)
        self.fc_logvar = Linear(hidden_dims[-1], latent_dim)
        # Decoder
        decoder_layers = [Linear(latent_dim, hidden_dims[-1]), activation_fn]
        hidden_dims.reverse()
        for i in range(1, len(hidden_dims)):
            decoder_layers += [Linear(hidden_dims[i-1], hidden_dims[i]), activation_fn]
        decoder_layers += [Linear(hidden_dims[-1], input_dim), Sigmoid()]  # Ensure final output is in [0, 1]
        self.decoder = Sequential(*decoder_layers)

    def encode(self, x):
        h = self.encoder(x)
        return self.fc_mu(h), self.fc_logvar(h)

    def reparameterize(self, mu, logvar):
        std = torch.exp(0.5 * logvar)
        eps = torch.randn_like(std)
        return mu + eps * std

    def decode(self, z):
        return self.decoder(z)

    def forward(self, x):
        mu, logvar = self.encode(x)
        z = self.reparameterize(mu, logvar)
        return self.decode(z), mu, logvar
    
    @staticmethod
    def set_seed(seed):
        torch.manual_seed(seed)
        np.random.seed(seed)
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False

# Defines Loss function for VAE which is calculated using reconstruction loss and KL Divergence
def vae_loss(recon_x, x, mu, logvar, loss_type='BCE'):
    if loss_type == 'BCE':
        recon_loss = binary_cross_entropy(recon_x, x.view(-1, recon_x.size(1)), reduction='sum')
    elif loss_type == 'MSE':
        recon_loss = mse_loss(recon_x, x.view(-1, recon_x.size(1)), reduction='sum')
    KLD = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())
    return recon_loss + KLD

# Training the VAE model using the training data
def train_vae(model, train_loader, optimizer, loss_type='BCE'):
    model.train()
    train_loss = 0
    for batch_idx, (data, _) in enumerate(train_loader):
        # Normalize data to be in the range [0, 1]
        data = data.float()  # Ensure data is float
        data = (data - data.min()) / (data.max() - data.min())
        optimizer.zero_grad()
        recon_batch, mu, logvar = model(data)
        loss = vae_loss(recon_batch, data, mu, logvar, loss_type)
        loss.backward()
        train_loss += loss.item()
        optimizer.step()
    return train_loss / len(train_loader.dataset)

# Training DoSE SVM model on the latent representation of the data obtained from the VAE model
def train_dose_svm(model, train_loader):
    model.eval()
    latents = []
    with torch.no_grad():
        for data, _ in train_loader:
            mu, _ = model.encode(data)
            latents.append(mu.cpu().numpy())
    latents = np.concatenate(latents, axis=0)
    svm_model = SGDOneClassSVM(random_state=42)
    whitened = WhitenedBenchmark(svm_model)
    whitened.fit(latents)
    return whitened

# DataLoader preparation
def get_data_loader(data, labels, batch_size=64):
    tensor_data = torch.tensor(data.values).float()
    tensor_labels = torch.tensor(labels.values).float()
    dataset = TensorDataset(tensor_data, tensor_labels)
    return DataLoader(dataset, batch_size=batch_size)
    #return DataLoader(dataset, batch_size=batch_size, shuffle=True)

# Generate predictions using the trained VAE model and the DoSE SVM model
def get_test_predictions(model, svm, test_loader):
    model.eval()
    predictions = []
    true_labels = []
    with torch.no_grad():
        for data, labels in test_loader:
            mu, _ = model.encode(data)
            mu = mu.cpu().numpy()
            svm_preds = svm.predict(mu)
            svm_preds = [0 if y == 1 else 1 for y in svm_preds]
            predictions.extend(svm_preds)
            true_labels.extend(labels.numpy())
    return predictions, true_labels


In [9]:
# NOTE: Approximately takes 5 minutes to run

# Create DataLoaders for training and testing
batch_size = 64
train_loader = get_data_loader(training_data, training_labels, batch_size)
test_loader = get_data_loader(test_data, test_labels, batch_size)

# Initialize the VAE model and optimizer
vae_model = VAE()
vae_model.set_seed(42)
optimizer = torch.optim.Adam(vae_model.parameters(), lr=0.001)

# Train the VAE model
num_epochs = 5  # Number of epochs
for epoch in range(num_epochs):
    loss = train_vae(vae_model, train_loader, optimizer)
    print(f"Epoch {epoch}: Loss = {loss}")

# Train the DoSE SVM model
dose_svm_model = train_dose_svm(vae_model, train_loader)

# Predict using the test data
test_predictions, true_labels = get_test_predictions(vae_model, dose_svm_model, test_loader)

# Print the results
print("Accuracy: ", accuracy_score(true_labels, test_predictions))
print("AUCROC: ", roc_auc_score(true_labels, test_predictions))
print("Classification Report:\n", classification_report(true_labels, test_predictions))

Epoch 0: Loss = 0.6435845868719934
Epoch 1: Loss = 0.623629160491492
Epoch 2: Loss = 0.6235394135073636
Epoch 3: Loss = 0.6235290576831732
Epoch 4: Loss = 0.6235514352574132
Accuracy:  0.9147099758158832
AUCROC:  0.7355800062161345
Classification Report:
               precision    recall  f1-score   support

         0.0       0.54      0.52      0.53     17508
         1.0       0.95      0.96      0.95    171459

    accuracy                           0.91    188967
   macro avg       0.75      0.74      0.74    188967
weighted avg       0.91      0.91      0.91    188967



## Method 5: XGBoost (Gradient Boosting Machine)
XGBoost is an algorithm for regression and classification which can be adapted for anomaly detection.

### Description
The XGBoost classifier gets intialized with the objective set to 'binary:logistic', which is used for binary (0 or 1) classification problems. The evaluation metric is set to 'auc', which stands for Area Under the ROC Curve, a common metric for binary classification tasks. The `use_label_encoder` parameter is set to False, which means that the labels are assumed to be already encoded as 0 or 1. The random state is set to 42 for reproducibility. Next, XGBoost classifier is trained using the `fit` method. Then using the `predict` method, the classifier generates predictions on the test data which are evaluated against the true labels of the test data.

### Performance
On testing, the model achieves AUCROC value of 0.970, significantly outperforming all other methods, including the ones described in the paper.
```
Accuracy:  0.9459799859234681
AUCROC:  0.9702319504954537
Classification Report:
               precision    recall  f1-score   support

           0       0.63      1.00      0.77     17508
           1       1.00      0.94      0.97    171459

    accuracy                           0.95    188967
   macro avg       0.82      0.97      0.87    188967
weighted avg       0.97      0.95      0.95    188967
```

In [9]:
# Initialize XGBoost classifier
xgb_classifier = xgb.XGBClassifier(objective='binary:logistic', eval_metric='auc', use_label_encoder=False, random_state=42)

# Train the model
xgb_classifier.fit(training_data, training_labels)

# Test the model
test_predictions = xgb_classifier.predict(test_data)

# Print the results
print("Accuracy: ", accuracy_score(test_labels, test_predictions))
print("AUCROC: ", roc_auc_score(test_labels, test_predictions))
print("Classification Report:\n", classification_report(test_labels, test_predictions))


Accuracy:  0.9459799859234681
AUCROC:  0.9702319504954537
Classification Report:
               precision    recall  f1-score   support

           0       0.63      1.00      0.77     17508
           1       1.00      0.94      0.97    171459

    accuracy                           0.95    188967
   macro avg       0.82      0.97      0.87    188967
weighted avg       0.97      0.95      0.95    188967

